# Train neural networks with `momics` and `tensorflow`

`momics` provides several useful resources to train neural networks with `tensorflow`. This notebook demonstrates how to train a simple neural network with `momics` and `tensorflow`.

## Connect to the data

First, we need to connect to the data. We will reuse the repository from the [previous tutorial](integrating-multiomics). 

In [1]:
from momics.momics import Momics

## Creating repository
repo = Momics("yeast_CNN_data.momics")

## Check that sequence and some tracks are registered
repo.seq()
repo.tracks()

,idx,label,path
0,0,ATAC,/data/momics/S288c_atac.bw
1,1,MNase,/data/momics/S288c_mnase.bw
2,2,ATAC_rescaled,tmp1ae0oz8z
3,3,MNase_rescaled,tmpxo8t09eu


## Modify some tracks

We can first pre-process the tracks to normalize them, and save them back to the local repository.

In [ ]:
import numpy as np

for track in ["ATAC", "MNase"]:
    cov = repo.tracks(track)
    q99 = np.nanpercentile(np.concatenate(list(cov.values())), 99)
    for chrom in cov.keys():
        arr = cov[chrom]
        arr = np.minimum(arr, q99)
        arr = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        cov[chrom] = np.nan_to_num(arr, nan=0)
    repo.ingest_track(cov, track + "_rescaled")

repo.tracks()

## Define model 

We will define a simple convolutional neural network with `tensorflow` to predict the target variable `ATAC` from the feature variable `MNase`. This requires to first define `MomicsDataset` objects for a training set, a validation set, and a test set. We will use `MNase` coverage scores over tiling genomic windows (width: 2048, stride: 2048) as feature variables to predict `ATAC` coverage scores over the same tiling genomic windows, but narrowed down to the 16bp around the center of the window.

In [2]:
from momics.dataset import MomicsDataset
import momics.utils as mutils

# Fetch data from the momics repository
features = "MNase_rescaled"
features_size = 2048
target = "ATAC_rescaled"
target_size = 16
stride = 2048
batch_size = 1024

bins = repo.bins(width=features_size, stride=stride, cut_last_bin_out=True)
bins_split, bins_test = mutils.split_ranges(bins, 0.8, shuffle=False)
bins_train, bins_val = mutils.split_ranges(bins_split, 0.8, shuffle=True)

train_dataset = MomicsDataset(repo, bins_train, features, target, target_size=target_size, batch_size=batch_size)
val_dataset = MomicsDataset(repo, bins_val, features, target, target_size=target_size, batch_size=batch_size)
test_dataset = MomicsDataset(repo, bins_test, features, target, target_size=target_size, batch_size=batch_size)

train_dataset  # noqa: B018
val_dataset  # noqa: B018
test_dataset

2024-10-21 09:57:51.409942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 09:57:51.421282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 09:57:51.424504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 09:57:51.432112: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 09:57:52.023569: W tensorflow/compiler/tf2

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 2048, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 16, 1), dtype=tf.float32, name=None))>

In [3]:
from momics.chromnn import ChromNN
import tensorflow as tf
from tensorflow.keras import layers  # type: ignore

model = ChromNN(
    layers.Input(shape=(features_size, 1)),
    layers.Dense(target_size, activation="linear"),
).model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 2048, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 2048, 32)  │        416 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 256, 32)   │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 32)   │        128 │ max_pooling1d[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256, 32)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 256, 32)   │      5,152 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 64, 32)    │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 32)    │        128 │ max_pooling1d_1[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 32)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 64, 32)    │      5,152 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 16, 32)    │          0 │ conv1d_2[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 32)    │        128 │ max_pooling1d_2[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16, 32)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 16, 16)    │      2,576 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16)    │         64 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16, 16)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 16, 16)    │      1,296 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16)    │         64 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 16, 16)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 32)    │          0 │ dropout_3[0][0],  │
│ (Concatenate)       │                   │            │ dropout_4[0][0] 

 Total params: 38,064 (148.69 KB)

 Trainable params: 37,744 (147.44 KB)

 Non-trainable params: 320 (1.25 KB)

## Fit the model 

In [4]:
import os
from pathlib import Path
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # type: ignore

os.makedirs(".chromnn", exist_ok=True)
callbacks_list = [
    CSVLogger(Path(".chromnn", "epoch_data.csv")),
    ModelCheckpoint(filepath=Path(".chromnn", "Checkpoint.keras").name, monitor="val_correlate", save_best_only=True),
    EarlyStopping(monitor="val_loss", patience=10, min_delta=1e-5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=6 // 2, min_lr=0.1 * 0.001),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=callbacks_list,
    steps_per_epoch=len(bins_train) // batch_size,
)

Epoch 1/20


I0000 00:00:1729497492.875502 2553417 service.cc:146] XLA service 0x7fe100009520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729497492.875515 2553417 service.cc:154]   StreamExecutor device (0): Quadro P400, Compute Capability 6.1
2024-10-21 09:58:12.942499: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-21 09:58:13.279690: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-21 09:58:18.083385: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.70GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-10-21 09:58:21.314785: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying alg

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step - loss: 2.2423

2024-10-21 09:58:34.701244: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.90GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-10-21 09:58:36.423690: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.58GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3/3 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - loss: 2.2155 - val_loss: 0.0260 - learning_rate: 0.0010
Epoch 2/20


...

2024-10-21 09:59:03.627357: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2024-10-21 09:59:03.627374: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5039837812499872075
2024-10-21 09:59:03.627383: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 18235436505003952707
/home/rsg/micromamba/envs/momics/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/home/rsg/micromamba/envs/momics/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - loss: 0.8577 - val_loss: 0.0286 - learning_rate: 1.0000e-04
Epoch 13/20


2024-10-21 09:59:04.299576: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5039837812499872075
/home/rsg/micromamba/envs/momics/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-10-21 09:59:04.299600: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 18235436505003952707
/home/rsg/micromamba/envs/momics/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_correlate available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 560ms/step - loss: 0.8691 - val_loss: 0.0293 - learning_rate: 1.0000e-04


2024-10-21 09:59:06.019402: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5039837812499872075
/home/rsg/micromamba/envs/momics/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-10-21 09:59:06.019431: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 18235436505003952707
/home/rsg/micromamba/envs/momics/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_correlate available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


## Evaluate and save model 

In [5]:
# Evaluate the model
model.evaluate(test_dataset)

# Save the model
model.save("chromnn_model.keras")

      1/Unknown 3s 3s/step - loss: 0.0323

2024-10-21 09:59:58.373818: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - loss: 0.0270


2024-10-21 09:59:59.142341: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5039837812499872075
2024-10-21 09:59:59.142361: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 18235436505003952707
/home/rsg/micromamba/envs/momics/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


## Use the model to predict ATAC-seq coverage

In [9]:
from momics.momicsquery import MomicsQuery

## Define 2048-bp-wide genomic windows, with a stride of 16 bp, and extract MNase data from it.
bb = repo.bins(width=features_size, stride=16, cut_last_bin_out=True)["XVI"]
dat = MomicsQuery(repo, bb).query_tracks(tracks=["MNase_rescaled"])
dat = np.array(list(dat.coverage["MNase_rescaled"].values()))
dat = np.nan_to_num(dat, nan=0)

## Run predictions
predictions = model.predict(dat)

## Export predictions as a bigwig
midpoints = (bb.Start + bb.End) // 2
bb.Start = (midpoints - target_size / 2).apply(int)
bb.End = (midpoints + target_size / 2).apply(int)
bb.to_bed("predictions_bins.bed")
mutils.pyranges_to_bw(bb, predictions, "predictions_cov.bw")

momics :: INFO :: 2024-10-21 10:02:21,179 :: Query completed in 0.2572s.


1848/1848 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
